In [1]:
import numpy as np
from collections import deque
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint

from coremdlr.viz import model_plots, CorePlotter
from coremdlr.models import NetworkModel
from coremdlr.datasets import FaciesDataset
from coremdlr.config.defaults import WELL_NAMES

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
wavenet_model_args = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : 20,
        'block_filters' : 128,
        'residual_filters' : 64,
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
    },
    'loss' : {'categorical_crossentropy': 0.5,
              'ordinal_squared_error': 0.5}
}

CHKPT_FNAME = 'temp_best_weights.h5'

wavenet_fit_args = {
    'batch_size' : 16,
    'epochs' : 15,
    'class_weighted' : True
}

names_deq = deque(WELL_NAMES)

In [3]:
results = {}

for i in range(len(names_deq) // 3):
    
    # Set up dataset, rotate well_names
    current_names = list(names_deq)
    fdset = FaciesDataset(current_names[:-3], test_wells=current_names[-3:],
                        features=["pseudoGR", "logs"],
                        pseudoGR_args={'features': ['mean', 'var'], 
                                                    'per_channel' : True},
                        label_resolution=32)
    
    names_deq.rotate(3)
    
    print(fdset)
    fdset.load_or_generate_data()
    
    # Build and train and re-load weights
    wnet_model = NetworkModel(fdset, model_args=wavenet_model_args)
    
    
    chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_acc', verbose=1,
                                    save_best_only=True, 
                                    save_weights_only=True)
    
    wavenet_fit_args['callbacks'] = [chkpt_callback]
    
    wnet_model.fit(fdset, **wavenet_fit_args)
    
    wnet_model.network.load_weights(CHKPT_FNAME)
    
    os.remove(CHKPT_FNAME)
    
    # Get test results
    for test_well in fdset.test_well_names:
        
        results[test_well] = wnet_model.preds_dataframe(test_well, logs=['GR', 'RDEP', 'PEF', 'SP'])

AssertionError: /home/administrator/Dropbox/core_data/facies/train_data/204-19-2_depth.npy does not exist!

In [ ]:
import pandas as pd

dfs = []
for well_name, results_df in results.items():
    results_df['well_name'] = well_name
    dfs.append(results_df)
    
df = pd.concat(dfs, ignore_index=True)
df

In [ ]:
df.to_csv('cross_test_wavenet_all_data.csv', index=False)

In [ ]:
axes = df.regression.hist(by=df.y_true, figsize=(15,15), alpha=0.4)
print(axes)

for i, ax in enumerate(axes.flat):
    df.regression[df.y_pred==i].hist(ax=ax, color='blue', alpha=0.4)

In [ ]:
axes = df.y_true.hist(by=df.well_name, figsize=(15,15), alpha=0.4)

for i, (ax, name) in enumerate(zip(axes.flat, df.WELL_NAME)):
    print(name)
    df.y_pred[df.well_name==name].hist(ax=ax, color='blue', alpha=0.4)

In [ ]:
fdset.well_names

In [ ]:
fdset.test_well_names

In [ ]:
np.arange(30).reshape(5, 3, 2)

In [ ]:
np.arange(30).reshape(5, 3, 2).reshape(-1, 2).reshape(5, 3 ,2)